# Smart Question Paper Generator (Advanced)

This notebook generates a question paper by:
1.  **OCR**: Using `nanonets/Nanonets-OCR2-3B` to extract text from previous years' papers.
2.  **Syllabus Analysis**: Extracting chapter-wise hours from the syllabus.
3.  **Weightage Calculation**: Combining syllabus hours and question frequency to prioritize topics.
4.  **Generation**: Using Groq's LLM to create a new paper based on these weights.

## 1. Setup and Dependencies

In [ ]:
# Install necessary libraries
# Note: 'flash_attn' is recommended for faster processing if you have a compatible GPU
!pip install PyPDF2 langchain langchain-groq python-dotenv transformers torch torchvision pillow accelerate bitsandbytes pdf2image

In [ ]:
import os
import re
import getpass
import torch
from PIL import Image
from pdf2image import convert_from_path
from transformers import AutoModelForCausalLM, AutoProcessor
from PyPDF2 import PdfReader
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

# Set up Groq API Key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 2. OCR with Nanonets (Local/Colab Model)

In [ ]:
# Load Nanonets Model
model_id = "nanonets/Nanonets-OCR2-3B"

try:
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto"
    )
    print("Nanonets model loaded successfully.")
except Exception as e:
    print(f"Failed to load model: {e}")
    print("Ensure you have enough VRAM (approx 8GB+ for 3B model in fp16).")

In [ ]:
def ocr_pdf(pdf_path):
    """Converts PDF to images and runs OCR on each page."""
    extracted_text = ""
    try:
        images = convert_from_path(pdf_path)
        print(f"Processing {len(images)} pages for {os.path.basename(pdf_path)}...")
        
        for i, image in enumerate(images):
            prompt = "<|image|>Extract the text from this document accurately into markdown format."
            inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
            
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=2048,
                do_sample=False  # Deterministic output usually better for OCR
            )
            
            generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            # Setup to just get the assistant response if the model includes prompt in output
            # (Adjust based on specific model behavior, some output prompt + completion)
            
            # Simple cleaning if prompt is repeated
            if prompt in generated_text:
                generated_text = generated_text.replace(prompt, "")
            
            extracted_text += f"--- Page {i+1} --- \n{generated_text}\n"
            
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    
    return extracted_text

## 3. Syllabus Parsing & Weightage Logic

In [ ]:
class SyllabusParser:
    def __init__(self, text):
        self.text = text
        self.modules = {}

    def parse_hours(self):
        # Regex to find "Module X ... Y Hours" or similar patterns
        # This needs to be adapted to the specific syllabus format
        # Pattern: look for "Module" identifier, then capture the Topic Name, then find "Hours" or "Hrs"
        
        # Example Pattern:  "1. Introduction ... 04 Hours"
        # We'll use a generic regex attempting to capture typical syllabus lines
        
        lines = self.text.split('\n')
        current_module = None
        
        for line in lines:
            # Heuristic: Check for lines ending in Hours/Hrs
            match = re.search(r'(\d+)(?:\s+)?(?:Hours|Hrs)', line, re.IGNORECASE)
            if match:
                hours = int(match.group(1))
                # Try to extract module name from the start of the line
                # Removing the hours part
                topic_name = line[:match.start()].strip()
                
                # Further clean topic name (remove leading numbers/bullets)
                topic_name = re.sub(r'^[\d\.\W]+', '', topic_name).strip()
                
                if topic_name and len(topic_name) > 3: # valid topic
                    self.modules[topic_name] = hours

        return self.modules

def calculate_weights(modules, previous_papers_text):
    # 1. Syllabus Weight (Normalized Hours)
    total_hours = sum(modules.values()) if modules else 1
    syllabus_weights = {k: v/total_hours for k, v in modules.items()}
    
    # 2. Frequency Weight (This is hard to do perfectly without NLP, using simple keyword matching)
    paper_text_lower = previous_papers_text.lower()
    frequency_counts = {}
    
    for topic in modules.keys():
        # Count occurrences of the topic or key terms in the previous papers
        # Taking first 2 words of topic as keywords
        keywords = topic.split()[:2]
        kw_regex = r"|".join([re.escape(k.lower()) for k in keywords if len(k) > 3])
        
        if kw_regex:
            count = len(re.findall(kw_regex, paper_text_lower))
            frequency_counts[topic] = count
        else:
            frequency_counts[topic] = 0
            
    total_freq = sum(frequency_counts.values()) if frequency_counts else 1
    freq_weights = {k: v/total_freq if total_freq > 0 else 0 for k, v in frequency_counts.items()}
    
    # 3. Combined Weight
    final_weights = {}
    for topic in modules:
        # 50% Hours, 50% Frequency
        final_weights[topic] = (syllabus_weights.get(topic, 0) * 0.5) + (freq_weights.get(topic, 0) * 0.5)
        
    return final_weights

## 4. Execution Pipeline

In [ ]:
# Main execution flow
qa_folder = "QA"
syllabus_text = ""
previous_papers_text = ""

if os.path.exists(qa_folder):
    for filename in os.listdir(qa_folder):
        file_path = os.path.join(qa_folder, filename)
        if filename.lower().endswith(".pdf"):
            print(f"Processing: {filename}")
            
            # Determine if syllabus or paper (simple check)
            is_syllabus = "syllabus" in filename.lower()
            
            # Use OCR for papers (better quality) or standard PDF read for syllabus if it's text-based
            # Assuming we want high quality for everything, let's use OCR for papers
            # For syllabus, often simple extraction works, but let's use OCR if we can afford the time
            
            if is_syllabus:
                # Syllabus might be text-based PDF, try simple extraction first for speed
                try:
                    reader = PdfReader(file_path)
                    text = "".join([p.extract_text() for p in reader.pages])
                    syllabus_text += text
                except:
                    # Fallback to OCR
                    syllabus_text += ocr_pdf(file_path)
            else:
                previous_papers_text += ocr_pdf(file_path)

# Parse Syllabus
parser = SyllabusParser(syllabus_text)
modules = parser.parse_hours()
print("Detected Modules & Hours:", modules)

# Calculate Weights
weights = calculate_weights(modules, previous_papers_text)
print("Calculated Topic Weights:", weights)

# Format High Importance Topics for Prompt
sorted_topics = sorted(weights.items(), key=lambda x: x[1], reverse=True)
top_focus_areas = "\n".join([f"- {t[0]} (Weight: {t[1]:.2f})" for t in sorted_topics[:5]])

## 5. Generation with Groq

In [ ]:
llm = ChatGroq(
    temperature=0.4,
    model_name="llama-3.3-70b-versatile"
)

prompt_text = """You are an expert question paper setter. Design a question paper that prioritizes the following High Importance Topics based on their syllabus weight and historical frequency:

**High Priority Topics:**
{top_focus_areas}

**Syllabus Context:**
{syllabus_snippet}

**Instructions:**
1. Allocate more marks/questions to the High Priority Topics listed above.
2. Ensure the paper covers the entire syllabus but skews difficulty/volume towards the weighted topics.
3. Follow the standard university pattern (e.g., Q1 Compulsory, Q2-Q6 with choices).
4. Create valid, conceptual, and application-based questions.
5. Output in clean Markdown.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a smart exam setter algorithm."),
    ("human", prompt_text)
])

chain = prompt | llm

try:
    response = chain.invoke({
        "top_focus_areas": top_focus_areas,
        "syllabus_snippet": syllabus_text[:15000] # Truncate for context limit
    })
    
    print("\n=== Generated Question Paper ===\n")
    final_paper = response.content
    print(final_paper)
    
    with open("Smart_Generated_Paper.md", "w", encoding="utf-8") as f:
        f.write(final_paper)
        
except Exception as e:
    print(f"Generation Failed: {e}")